#  Home assignment #2. Particle filter

In this homework assignment, we will implement an algorithm for estimating the robot's pose known as a particle filter.
A particle filter consists of the following steps:
1. The movement of particles in accordance with the kinematic motion model. The movement is carried out using the probabilistic motion function, taking into account the randomness of the process (noise of the motion).
2. Comparison of the obtained measurement with the expected one by applying a probabilistic measurement function that estimates the probability of the obtained measurement given a fixed position of the particle. As a result of this step, each particle is assigned a weight proportional to the likelihood of the measurement given the position of the particle.
3. Resampling is a process in which the probability of a particle to be chosen for a new set is proportional to the weight (likelihood) of the particle.


## Task 1
 
Implement the motion model of the mobile robot $ p (x_ {t + 1} | x_t, u_t) $ as part of the *Robot* class. The method must be named *move()*. The *move()* method takes as input the current position of the robot and a vector of control signals (the steering angle ($ \alpha $) and the distance the robot should move ($ d $)). The *move()* method must return an instance of the *Robot* class with a new state vector $ (x, y, \theta) $.

For this homework assignment, we will assume that our robot has car-like kinematics. Such kinematics is described by the [bicycle model] (https://nabinsharma.wordpress.com/2014/01/02/kinematics-of-a-robot-bicycle-model/) (you can also refer to Lecture 3. The tricycle (note the difference in the notation). You need to implement a bicycle model in the *move()* method as an approximation of car-like kinematics.

**Important:** The coordinates $ (x, y) $ of the robot's state vector set the position of the center of the rear wheel axle of the robot.

**Important:** The *move()* method must also simulate the noise of the control signals. For this, additive normal noise is applied to each component of the control signal vector. The steering angle noise is specified by the *Robot.steering_noise* class attribute. The movement noise is specified by the *Robot.distance_noise* class attribute. The *Robot.steering_noise* and *Robot.distance_noise* parameters set the standard deviation ($ \sigma $) of the normal distribution.

In [72]:
from math import *
import random

In [73]:
# --------
# 
# the "world" has 4 landmarks.
# the robot's initial coordinates are somewhere in the square
# represented by the landmarks.

landmarks  = [[100.0, 0.0], [0.0, 0.0], [0.0, 100.0], [100.0, 100.0]] # position of 4 landmarks
world_size = 100.0 # world is NOT cyclic. Robot is allowed to travel "out of bounds"

In [74]:
class Robot:

    # --------

    # init: 
    #   creates robot and initializes location/orientation 
    #

    def __init__(self, length = 10.0):
        self.x = random.random() * world_size # initial x position
        self.y = random.random() * world_size # initial y position
        self.orientation = random.random() * 2.0 * pi # initial orientation
        self.length = length # length of robot
        self.bearing_noise  = 0.0 # initialize bearing noise to zero
        self.distance_noise = 0.0 # initialize distance noise to zero
        self.steering_noise = 0.0 # initialize steering noise to zero

    
    def __repr__(self):
        return '[x=%.6s y=%.6s theta=%.6s]' % (str(self.x), str(self.y), str(self.orientation))
    # --------
    # set: 
    #   sets a robot coordinate
    #

    def set(self, new_x, new_y, new_orientation):

        if new_orientation < 0 or new_orientation >= 2 * pi:
            raise ValueError('Orientation must be in [0..2pi]')
        self.x = float(new_x)
        self.y = float(new_y)
        self.orientation = float(new_orientation)


    # --------
    # set_noise: 
    #	sets the noise parameters
    #

    def set_noise(self, new_b_noise, new_s_noise, new_d_noise):
        # makes it possible to change the noise parameters
        # this is often useful in particle filters
        self.bearing_noise  = float(new_b_noise)
        self.steering_noise = float(new_s_noise)
        self.distance_noise = float(new_d_noise)
    
    ############# ONLY ADD/MODIFY CODE BELOW ###################

    # --------
    # move:
    #   move along a section of a circular path according to motion parameters
    #
    
    def move(self, motion):
        import numpy as np
        alpha = np.random.normal(motion[0], self.steering_noise) 
        distance = np.random.normal(motion[1], self.distance_noise)
        
        if alpha == 0:
            X = self.x + distance * cos(self.orientation)
            Y = self.y + distance * sin(self.orientation)
        else:
            R = self.length / tan(alpha)
            beta = distance / R

            Xc = self.x - R * sin(self.orientation)
            Yc = self.y + R * cos(self.orientation)
            self.orientation += beta
            X = Xc + R * sin(self.orientation)
            Y = Yc - R * cos(self.orientation)
            self.orientation %= 2 * pi
        self.set(X, Y, self.orientation)
        return self
 
    ############## ONLY ADD/MODIFY CODE ABOVE ####################

In [75]:
## --------
## TEST CASE #1:
## 
## The following code should print:
##       Robot:     [x=0.0 y=0.0 theta=0.0]
##       Robot:     [x=10.0 y=0.0 theta=0.0]
##       Robot:     [x=19.861 y=1.4333 theta=0.2886]
##       Robot:     [x=39.034 y=7.1270 theta=0.2886]
##

length = 20.
bearing_noise  = 0.0
steering_noise = 0.0
distance_noise = 0.0

myrobot = Robot(length)
myrobot.set(0.0, 0.0, 0.0)
myrobot.set_noise(bearing_noise, steering_noise, distance_noise)

motions = [[0.0, 10.0], [pi / 6.0, 10], [0.0, 20.0]]

T = len(motions)

print('Robot:    ', myrobot)
for t in range(T):
    myrobot = myrobot.move(motions[t])
    print('Robot:    ', myrobot)

Robot:     [x=0.0 y=0.0 theta=0.0]
Robot:     [x=10.0 y=0.0 theta=0.0]
Robot:     [x=19.861 y=1.4333 theta=0.2886]
Robot:     [x=39.034 y=7.1270 theta=0.2886]


In [76]:
## --------
## TEST CASE #2:
## 
## The following code should print:
##      Robot:     [x=0.0 y=0.0 theta=0.0]
##      Robot:     [x=9.9828 y=0.5063 theta=0.1013]
##      Robot:     [x=19.863 y=2.0201 theta=0.2027]
##      Robot:     [x=29.539 y=4.5259 theta=0.3040]
##      Robot:     [x=38.913 y=7.9979 theta=0.4054]
##      Robot:     [x=47.887 y=12.400 theta=0.5067]
##      Robot:     [x=56.369 y=17.688 theta=0.6081]
##      Robot:     [x=64.273 y=23.807 theta=0.7094]
##      Robot:     [x=71.517 y=30.695 theta=0.8108]
##      Robot:     [x=78.027 y=38.280 theta=0.9121]
##      Robot:     [x=83.736 y=46.485 theta=1.0135]

length = 20.
bearing_noise  = 0.0
steering_noise = 0.0
distance_noise = 0.0

myrobot = Robot(length)
myrobot.set(0.0, 0.0, 0.0)
myrobot.set_noise(bearing_noise, steering_noise, distance_noise)

motions = [[0.2, 10.] for row in range(10)]

T = len(motions)

print('Robot:    ', myrobot)
for t in range(T):
    myrobot = myrobot.move(motions[t])
    print('Robot:    ', myrobot) 

Robot:     [x=0.0 y=0.0 theta=0.0]
Robot:     [x=9.9828 y=0.5063 theta=0.1013]
Robot:     [x=19.863 y=2.0201 theta=0.2027]
Robot:     [x=29.539 y=4.5259 theta=0.3040]
Robot:     [x=38.913 y=7.9979 theta=0.4054]
Robot:     [x=47.887 y=12.400 theta=0.5067]
Robot:     [x=56.369 y=17.688 theta=0.6081]
Robot:     [x=64.273 y=23.807 theta=0.7094]
Robot:     [x=71.517 y=30.695 theta=0.8108]
Robot:     [x=78.027 y=38.280 theta=0.9121]
Robot:     [x=83.736 y=46.485 theta=1.0135]


## Task 2
 
Implement a mobile robot measurement method. The method must be named *sense()*. The *sense()* method takes as input the current state of the robot (*self*) and returns $ z $ - the current measurement consisting of four bearings to four landmarks located in space. Bearing is the angle at which the object is observed from the current position. The angle at which the robot observes each landmark is measured from the robot's current orientation $ \theta $. The counterclockwise direction is assumed to be positive.

To calculate the bearing, you need the position of the landmarks in space. It is set by the global variable *landmarks*.

**Important:** The *sense()* method should also simulate the measurement noise. For this, additive normal noise is applied to each component of the measurement vector. The measurement noise is specified by the *Robot.bearing_noise* class attribute. This parameter specifies the standard deviation ($ \sigma $) of the normal distribution. Provide the ability to calculate a noisy measurement vector by passing an input argument *no_noise = True* to the function.

Copy the *Robot* class to the cell below and add the *sense()* method to it.

In [77]:
class Robot:

    # --------

    # init: 
    #   creates robot and initializes location/orientation 
    #

    def __init__(self, length = 10.0):
        self.x = random.random() * world_size # initial x position
        self.y = random.random() * world_size # initial y position
        self.orientation = random.random() * 2.0 * pi # initial orientation
        self.length = length # length of robot
        self.bearing_noise  = 0.0 # initialize bearing noise to zero
        self.distance_noise = 0.0 # initialize distance noise to zero
        self.steering_noise = 0.0 # initialize steering noise to zero

    
    def __repr__(self):
        return '[x=%.6s y=%.6s theta=%.6s]' % (str(self.x), str(self.y), str(self.orientation))
    # --------
    # set: 
    #   sets a robot coordinate
    #

    def set(self, new_x, new_y, new_orientation):

        if new_orientation < 0 or new_orientation >= 2 * pi:
            raise ValueError('Orientation must be in [0..2pi]')
        self.x = float(new_x)
        self.y = float(new_y)
        self.orientation = float(new_orientation)


    # --------
    # set_noise: 
    #	sets the noise parameters
    #

    def set_noise(self, new_b_noise, new_s_noise, new_d_noise):
        # makes it possible to change the noise parameters
        # this is often useful in particle filters
        self.bearing_noise  = float(new_b_noise)
        self.steering_noise = float(new_s_noise)
        self.distance_noise = float(new_d_noise)
    
    ############# ONLY ADD/MODIFY CODE BELOW ###################

    # --------
    # move:
    #   move along a section of a circular path according to motion parameters
    #
    
    def move(self, motion):
        import numpy as np
        alpha = np.random.normal(motion[0], self.steering_noise) 
        distance = np.random.normal(motion[1], self.distance_noise)
        
        if alpha == 0:
            X = self.x + distance * cos(self.orientation)
            Y = self.y + distance * sin(self.orientation)
        else:
            R = self.length / tan(alpha)
            beta = distance / R

            Xc = self.x - R * sin(self.orientation)
            Yc = self.y + R * cos(self.orientation)
            self.orientation += beta
            X = Xc + R * sin(self.orientation)
            Y = Yc - R * cos(self.orientation)
            self.orientation %= 2 * pi
        self.set(X, Y, self.orientation)
        return self
    
    def sense(self, no_noise=True):
        bearings = list()
        x = landmarks[0][0]
        y = landmarks[0][1]
        #add 2pi
        bearings.append(2 * pi + atan((self.y - y) / (self.x - x)) - self.orientation)
        
        x = landmarks[1][0]
        y = landmarks[1][1]
        
        #add pi
        bearings.append(pi + atan((self.y - y) / (self.x - x)) - self.orientation)
        
        x = landmarks[2][0]
        y = landmarks[2][1]
        #add pi
        bearings.append(pi + atan((self.y - y) / (self.x - x)) - self.orientation)
        
        x = landmarks[3][0]
        y = landmarks[3][1]
        
        #add nothing
        bearings.append(atan((self.y - y) / (self.x - x)) - self.orientation)
        if no_noise:
            return bearings
        else:
            for i in range(0, len(bearings)):
                bearings[i] = np.random.normal(bearings[i], self.bearing_noise)
            return bearings
    ############## ONLY ADD/MODIFY CODE ABOVE ####################

In [78]:
## --------
## TEST CASE #1:
## 
## 1) The following code should print the list:
## [6.004885648174475, 3.7295952571373605, 1.9295669970654687, 0.8519663271732721]

length = 20.
bearing_noise  = 0.0
steering_noise = 0.0
distance_noise = 0.0

myrobot = Robot(length)
myrobot.set(30.0, 20.0, 0.0)
myrobot.set_noise(bearing_noise, steering_noise, distance_noise)

print('Robot:        ', myrobot) 
print('Measurements: ', myrobot.sense()) 

Robot:         [x=30.0 y=20.0 theta=0.0]
Measurements:  [6.004885648174475, 3.7295952571373605, 1.9295669970654687, 0.851966327173272]


In [79]:
## --------
## TEST CASE #2:
## 
## 2) The following code should print the list^
## [5.376567117456516, 3.101276726419402, 1.3012484663475101, 0.22364779645531352]

length = 20.
bearing_noise  = 0.0
steering_noise = 0.0
distance_noise = 0.0

myrobot = Robot(length)
myrobot.set(30.0, 20.0, pi / 5.0)
myrobot.set_noise(bearing_noise, steering_noise, distance_noise)

print('Robot:        ', myrobot) 
print('Measurements: ', myrobot.sense()) 

Robot:         [x=30.0 y=20.0 theta=0.6283]
Measurements:  [5.376567117456516, 3.101276726419402, 1.3012484663475101, 0.2236477964553134]


## Task 3

Implement the mobile robot observation model $ p (z_t | x_t, M) $. The method must be named *measurement_prob()*. The *measurement_prob()* method takes a measurement vector $ z $ as input and returns the likelihood of the measurement. Likelihood is calculated as the product of four (by the number of landmarks) normal distributions of measurement errors. Each normal distribution shows the bearing probability i.e. the normal distribution of the error for each bearing has the mathematical expectation in the true (expected) bearing value and the variance given by the *Robot.bearing_noise* parameter.

**Important:** Remember to normalize the angles when calculating bearing errors. The error must be in the range $ - \pi ... + \pi $.

**Important:** To get the true (expected) values of the measurements, you can use the *sense()* method with the *no_noise = True* flag.

Copy the *Robot* class into the cell below and add the *measurement_prob()* method to it.

In [80]:
import scipy.stats as st
import numpy as np

class Robot:

    # --------

    # init: 
    #   creates robot and initializes location/orientation 
    #

    def __init__(self, length = 10.0):
        self.x = random.random() * world_size # initial x position
        self.y = random.random() * world_size # initial y position
        self.orientation = random.random() * 2.0 * pi # initial orientation
        self.length = length # length of robot
        self.bearing_noise  = 0.0 # initialize bearing noise to zero
        self.distance_noise = 0.0 # initialize distance noise to zero
        self.steering_noise = 0.0 # initialize steering noise to zero

    
    def __repr__(self):
        return '[x=%.6s y=%.6s theta=%.6s]' % (str(self.x), str(self.y), str(self.orientation))
    # --------
    # set: 
    #   sets a robot coordinate
    #

    def set(self, new_x, new_y, new_orientation):

        if new_orientation < 0 or new_orientation >= 2 * pi:
            raise ValueError('Orientation must be in [0..2pi]')
        self.x = float(new_x)
        self.y = float(new_y)
        self.orientation = float(new_orientation)


    # --------
    # set_noise: 
    #	sets the noise parameters
    #

    def set_noise(self, new_b_noise, new_s_noise, new_d_noise):
        # makes it possible to change the noise parameters
        # this is often useful in particle filters
        self.bearing_noise  = float(new_b_noise)
        self.steering_noise = float(new_s_noise)
        self.distance_noise = float(new_d_noise)
    
    ############# ONLY ADD/MODIFY CODE BELOW ###################

    # --------
    # move:
    #   move along a section of a circular path according to motion parameters
    #
    
    def move(self, motion):
        alpha = np.random.normal(motion[0], self.steering_noise) 
        distance = np.random.normal(motion[1], self.distance_noise)
        
        if alpha == 0:
            X = self.x + distance * cos(self.orientation)
            Y = self.y + distance * sin(self.orientation)
        else:
            R = self.length / tan(alpha)
            beta = distance / R

            Xcorr = self.x - R * sin(self.orientation)
            Ycorr = self.y + R * cos(self.orientation)
            self.orientation += beta
            X = Xcorr + R * sin(self.orientation)
            Y = Ycorr - R * cos(self.orientation)
            self.orientation %= 2 * pi
            print(Xcorr)
            print(Ycorr)
        self.set(X, Y, self.orientation)
        return self
    
    def sense(self, no_noise=True):
        bearings = list()
        x = landmarks[0][0]
        y = landmarks[0][1]
        #add 2pi
        bearings.append(2 * pi + atan((self.y - y) / (self.x - x)) - self.orientation)
        
        x = landmarks[1][0]
        y = landmarks[1][1]
        #add pi
        bearings.append(pi + atan((self.y - y) / (self.x - x)) - self.orientation)
        
        x = landmarks[2][0]
        y = landmarks[2][1]
        #add pi
        bearings.append(pi + atan((self.y - y) / (self.x - x)) - self.orientation)
        
        x = landmarks[3][0]
        y = landmarks[3][1]
        #add nothing
        bearings.append(atan((self.y - y) / (self.x - x)) - self.orientation)
        if no_noise:
            return bearings
        else:
            for i in range(0, len(bearings)):
                bearings[i] = np.random.normal(bearings[i], self.bearing_noise)
            return bearings
        
    def measurement_prob(self, z):
        
        error = 1.0
        predicted_measurements = self.sense(no_noise=True)
        
        for i in range(len(z)):
            error_bearing = abs(z[i] - predicted_measurements[i])
            error_bearing = (error_bearing + pi) % (2.0 * pi) - pi # truncate (-pi pi)

            # update Gaussian
            error *= (exp(- (error_bearing ** 2) / (self.bearing_noise ** 2) / 2.0) /  
                     sqrt(2.0 * pi * (self.bearing_noise ** 2)))
        
        return error


## Task 4

Run the particle filter based on the *Robot* class you have implemented. Add a step-by-step visualization of the particle filter for the second test case. The visualization should reflect:
1. Map with marked positions of landmarks.
2. Particles - it is enough to reflect only $ (x, y) $.
3. The final estimated position of the robot at each moment in time.

In [81]:
from math import *
import random

max_steering_angle = pi / 4.0 # You do not need to use this value, but keep in mind the limitations of a real car.
bearing_noise = 0.1 
steering_noise = 0.1
distance_noise = 5.0

tolerance_xy = 15.0 # Tolerance for localization in the x and y directions.
tolerance_orientation = 0.25 # Tolerance for orientation.


# --------
# 
# the "world" has 4 landmarks.
# the robot's initial coordinates are somewhere in the square
# represented by the landmarks.

landmarks  = [[100.0, 0.0], [0.0, 0.0], [0.0, 100.0], [100.0, 100.0]] # position of 4 landmarks
world_size = 100.0 # world is NOT cyclic. Robot is allowed to travel "out of bounds"

In [82]:
# Some utility functions

def get_position(p):
    x = 0.0
    y = 0.0
    orientation = 0.0
    for i in range(len(p)):
        x += p[i].x
        y += p[i].y
        # orientation is tricky because it is cyclic. By normalizing
        # around the first particle we are somewhat more robust to
        # the 0=2pi problem
        orientation += (((p[i].orientation - p[0].orientation + pi) % (2.0 * pi)) 
                        + p[0].orientation - pi)
    return [x / len(p), y / len(p), orientation / len(p)]


# The following code generates ground truth poses and measurements
def generate_ground_truth(motions):

    myrobot = Robot()
    myrobot.set_noise(bearing_noise, steering_noise, distance_noise)

    Z = []
    T = len(motions)

    for t in range(T):
        myrobot = myrobot.move(motions[t])
        Z.append(myrobot.sense())
    #print 'Robot:    ', myrobot
    return [myrobot, Z]


# The following code prints the measurements associated
# with generate_ground_truth
def print_measurements(Z):

    T = len(Z)

    print ('measurements = [[%.8s, %.8s, %.8s, %.8s],' % \
        (str(Z[0][0]), str(Z[0][1]), str(Z[0][2]), str(Z[0][3])))
    for t in range(1,T-1):
        print ('                [%.8s, %.8s, %.8s, %.8s],' % \
            (str(Z[t][0]), str(Z[t][1]), str(Z[t][2]), str(Z[t][3])))
    print ('                [%.8s, %.8s, %.8s, %.8s]]' % \
        (str(Z[T-1][0]), str(Z[T-1][1]), str(Z[T-1][2]), str(Z[T-1][3])))


# The following code checks to see if your particle filter
# localizes the robot to within the desired tolerances
# of the true position. The tolerances are defined at the top.
def check_output(final_robot, estimated_position):

    error_x = abs(final_robot.x - estimated_position[0])
    error_y = abs(final_robot.y - estimated_position[1])
    error_orientation = abs(final_robot.orientation - estimated_position[2])
    error_orientation = (error_orientation + pi) % (2.0 * pi) - pi
    correct = error_x < tolerance_xy and error_y < tolerance_xy \
              and error_orientation < tolerance_orientation
    return correct

In [83]:
def particle_filter(motions, measurements, N=1500): # We will use 500 particles
    # --------
    #
    # Make particles
    # 

    particles = []
    for i in range(N):
        r = Robot()
        r.set_noise(bearing_noise, steering_noise, distance_noise)
        particles.append(r)

    # --------
    #
    # Update particles
    #     

    for t in range(len(motions)):
    
        # motion update (prediction)
        particles_after_motion = []
        for i in range(N):
            particles_after_motion.append(particles[i].move(motions[t]))
        particles = particles_after_motion

        # measurement update (correction)
        weights = []
        for i in range(N):
            weights.append(particles[i].measurement_prob(measurements[t]))

        # resampling
        particles_resampled = []
        index = int(random.random() * N)
        beta = 0.0
        print (weights)
        mw = max(weights)
        for i in range(N):
            beta += random.random() * 2.0 * mw
            while beta > weights[index]:
                beta -= weights[index]
                index = (index + 1) % N
            r = Robot()
            r.set(particles[index].x, particles[index].y, particles[index].orientation)
            r.set_noise(bearing_noise, steering_noise, distance_noise)
            particles_resampled.append(r)
        particles = particles_resampled
    
    return get_position(particles)

In [84]:
## --------
## TEST CASE #1:
## 
##1) Calling the particle_filter function with the following
##    motions and measurements should return a [x,y,orientation]
##    vector near [x=93.476 y=75.186 orient=5.2664], that is, the
##    robot's true location.
##
motions = [[2. * pi / 10, 20.] for row in range(8)]
measurements = [[4.746936, 3.859782, 3.045217, 2.045506],
                [3.510067, 2.916300, 2.146394, 1.598332],
                [2.972469, 2.407489, 1.588474, 1.611094],
                [1.906178, 1.193329, 0.619356, 0.807930],
                [1.352825, 0.662233, 0.144927, 0.799090],
                [0.856150, 0.214590, 5.651497, 1.062401],
                [0.194460, 5.660382, 4.761072, 2.471682],
                [5.717342, 4.736780, 3.909599, 2.342536]]

print (particle_filter(motions, measurements))
x = generate_ground_truth(motions)
print(x[0])

[5.15076640244872e-159, 0.0, 4.821324439040021e-38, 0.0, 7.9018123184987e-238, 0.0, 0.0, 0.0, 3.977577489175815e-100, 6.018346523739907e-82, 0.0, 3.1193481857682384e-271, 0.0, 0.0, 4.697292596266765e-295, 0.0, 4.7667644890127185e-175, 0.0, 0.0, 0.0, 0.0, 7.160734791693563e-138, 0.0, 4.146252586881703e-260, 1.3537101528585893e-126, 1.1242796696680138e-132, 2.56130662121244e-15, 6.997153294235484e-266, 3.588379034283703e-203, 2.9836290206019692e-276, 0.0, 1.1008830745667152e-20, 5e-324, 3.157148558656947e-70, 0.0, 4.877955478936856e-163, 4.1151529743504317e-221, 2.7960345458932463e-126, 3.92201e-318, 1.12458549737e-311, 2.968597651214198e-278, 0.0, 0.0, 0.0, 6.736357432735532e-101, 1.3820163328753816e-101, 6.037335896061312e-205, 0.0, 1.4390893733021057e-284, 8.83894041137652e-253, 0.0, 8.903040728671855e-279, 3.008337273467459e-276, 6.327579982800086e-156, 3.612136137100641e-306, 5.936096753267614e-162, 0.0, 0.0, 2.710030232174647e-237, 1.6481176384232392e-226, 1.6409026559633624e-203, 

[0.0, 3.368612124248406e-287, 0.0, 0.0, 0.0, 9.490598342253785e-221, 3.4711818111195718e-251, 1.1927960895473964e-298, 6.727036201251574e-255, 9.910001957435562e-264, 1.725686956307394e-284, 0.0, 0.0, 4.5972153529619995e-294, 8.40919189061015e-251, 0.0, 2.1041633022991768e-227, 2.5e-323, 0.0, 3.014492950066585e-287, 1.6113491900099515e-230, 5e-323, 0.0, 5.1817063470336144e-210, 2.4258063784905923e-261, 2.0775955e-315, 0.0, 4.269964375e-315, 6.539618553142528e-223, 0.0, 8.261697297483233e-222, 7.757445607187814e-263, 4.037891005408463e-286, 7.375940875431347e-282, 0.0, 0.0, 0.0, 1.5e-323, 0.0, 1e-323, 6.044439387560825e-233, 0.0, 0.0, 0.0, 1.504441520417946e-261, 2.8511764018143333e-306, 9.561549412237882e-291, 8.171072160465307e-210, 4.895351090331361e-277, 0.0, 0.0, 4.027250519440876e-272, 3.82320136e-316, 6.220850834346797e-246, 1.9002967270505513e-242, 0.0, 8.546877895829434e-208, 3.651432122593068e-256, 2.863944580110802e-207, 0.0, 6.191270001607314e-296, 3.620504942680497e-269, 8.

[0.0, 2.6097e-320, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.8616197236281315e-303, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.63513866e-315, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.151687204678629e-295, 0.0, 0.0, 6.642269022063171e-184, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.384083271755e-312, 0.0, 0.0, 0.0, 0.0, 0.0, 7.1966e-320, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.73e-321, 0.0, 9.676263e-317, 0.0, 0.0, 4.1967086138958425e-280, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5e-324, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.249253573540855e-300, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.1550628e-316, 0.0, 0.0, 0.0, 4.3227633847020326e-285, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.172744005209441e-283, 0.0, 0.0, 0.0, 0.0, 2.5787903956462625e-308, 0.0, 0.0, 0.0, 0.0,

[2.946250629997741e-284, 3.1028337845188094e-271, 0.0, 3.2686919555134763e-165, 4.626360768169254e-292, 6.537716477688269e-283, 0.0, 1.7076368409899806e-279, 2.5660827166896877e-277, 6.146564831160939e-269, 3.0258116510818557e-243, 0.0, 9.074623018512547e-281, 5.3430865813949295e-247, 1.7425665832808296e-302, 9.040004602840333e-263, 8.99963095527395e-298, 1.9356459995616252e-266, 2.183583386960246e-274, 8.355504498880419e-269, 1.24e-322, 2.959965884331536e-269, 0.0, 1.300271074714669e-278, 2.041521348715955e-298, 2.138789e-317, 1.8332403432507186e-270, 3.363234076858864e-288, 2.6324808194739303e-290, 0.0, 1.7010600970377885e-299, 8.75215592642591e-251, 1.1162420957673108e-286, 9.852116330540094e-260, 8.436742628403019e-249, 5.289857492295985e-274, 7.7425071128061e-310, 1.6724831606457746e-236, 9.7529715e-317, 2.741254125824044e-241, 1.3236e-320, 3.098350718027053e-275, 0.0, 1.3596585126594804e-274, 2.7100183453354122e-258, 0.0, 3.4785722870606154e-194, 3.0769287788722115e-254, 0.0, 1.1

[1.5591392691181349e-235, 1.5284578116927923e-200, 9.063006726381149e-185, 1.8968258895245332e-204, 2.701549768573907e-173, 7.264119947245353e-196, 1.3177965657525313e-202, 1.8322698553657512e-192, 4.162030596082089e-186, 1.621060814712959e-230, 7.12923033850448e-305, 5.859009782029965e-184, 1.5082278665779902e-188, 1.4834689876185233e-272, 1.2513694097695496e-225, 1.038563561789226e-264, 1.13937e-318, 1.0856690002912393e-269, 1.4758648622983765e-235, 2.4894539835729607e-264, 4.896825923488018e-212, 2.0076562002419587e-193, 1.5337315794877177e-250, 1.2500499904432322e-197, 1.3324074927628002e-194, 2.729769954706619e-209, 1.2923466503151497e-237, 5.652789192607475e-244, 3.1930003854825616e-191, 3.974088339597148e-236, 3.3992592185865347e-221, 4.083686682126531e-203, 1.4425929718182502e-185, 1.2226448972584961e-228, 2.4126086859524688e-191, 2.974538770598202e-191, 8.665516440845541e-185, 6.5794229404219805e-198, 2.806125894711933e-191, 6.392479362278621e-191, 0.0, 1.176458743243797e-250,

[0.0, 0.0, 8.682432383033466e-238, 1.3481575353526086e-226, 1.53851813965e-313, 6.890657719959591e-158, 1.9610937594546236e-246, 4.4205874398147835e-218, 3.0930735809146343e-235, 1.4423248423734735e-112, 8.674016907109097e-102, 3.3384510300448953e-202, 0.0, 2.0283764732172535e-203, 1.2177679367426412e-264, 4.980944675546667e-236, 2.3956936686624108e-127, 1.3704581365912296e-297, 3.5331708403685325e-144, 9.062497261735901e-236, 1.6790355942716274e-236, 0.0, 1.8726278254514365e-222, 5.747123690018278e-95, 1.923166283579488e-253, 1.2636682276671547e-141, 6.495327819512236e-247, 1.906782954855432e-126, 2.6489095900750904e-136, 5.07348906398241e-257, 3.4055163222851416e-160, 9.709703895567901e-240, 2.50837600652663e-133, 1.4491919874952734e-258, 1.3183521734426526e-184, 1.907379914729068e-281, 8.268022532123747e-175, 0.0, 2.264321530427024e-118, 2.8073136988846446e-108, 0.0, 3.4267020597511435e-151, 9.944569247317303e-112, 2.055377494149441e-189, 4.145048740339509e-136, 2.722839575325266e-1

[3.339690194714975e-148, 2.353678107653708e-05, 7.607277803317794e-37, 6.11768026307525e-71, 8.102855787962552e-147, 1.7740823853261715e-181, 1.3574814515787213e-25, 3.148558431891595e-86, 3.199557405362644e-14, 2.0999559235984765e-27, 4.5366242862535456e-38, 1.0818138300037567e-143, 1.3490611720890747e-164, 3.233019658867644e-82, 4.510104991136409e-102, 7.212497048477947e-63, 2.8142512598010994e-08, 3.2769991075176935e-53, 1.5485426492060503e-11, 5.818015755798844e-48, 7.61077483892511e-73, 3.0469984032119954e-61, 4.825581265104009e-100, 2.242495265458626e-118, 1.2115911027204534e-13, 2.6520531098449707e-54, 4.116177171438454e-105, 1.1243397012716471e-56, 2.0522458713775507e-10, 5.092542895281944e-180, 2.547642282547135e-112, 2.1382107550359815e-47, 3.0986941917903015e-30, 7.699473599294362e-118, 1.6615225247210225e-91, 3.7518279669364015e-06, 9.550205076003125e-15, 7.474456689368684e-43, 4.5790801549586124e-38, 1.0436749024872384e-15, 6.461547911648966e-158, 3.632620898701174e-66, 2.

In [85]:
## --------
## TEST CASE #1:
##
## 2) You can generate your own test cases by generating
##    measurements using the generate_ground_truth function.
##    It will print the robot's last location when calling it.
##

number_of_iterations = 6
motions = [[2. * pi / 20, 12.] for row in range(number_of_iterations)]

x = generate_ground_truth(motions)
final_robot = x[0]
measurements = x[1]
estimated_position = particle_filter(motions, measurements)
print_measurements(measurements)
print('Ground truth:    ', final_robot) 
print('Particle filter: ', estimated_position)
print('Code check:      ', check_output(final_robot, estimated_position)) 

[0.0, 2.281726464607322e-141, 5.411905007067878e-186, 6.8877719891889685e-223, 1.8609933432006463e-148, 0.0, 1.4348176535417204e-267, 0.0, 2.77063598907263e-310, 2.7947186131490865e-208, 1.7921811982708586e-115, 4.9986551074385634e-217, 5.31240243897236e-198, 3.70546923436707e-310, 1.0720100097851447e-213, 5.953477101805943e-253, 0.0, 2.4537783676135553e-216, 9.908573981078772e-187, 4.8832444895655025e-149, 0.0, 0.0, 2.5552992218095583e-168, 0.0, 2.8177023862168105e-221, 1.2009968209717425e-289, 0.0, 6.156155344700972e-197, 1.3635049366910711e-217, 3.5787656049741e-219, 6.9e-322, 0.0, 2.381476368413754e-266, 1.5382752722372866e-235, 2.757297235207112e-259, 8.474322182435819e-191, 6.119250665850425e-249, 4.3463200904392585e-132, 4.888314131444692e-200, 4.398796804964351e-213, 0.0, 3.395003244746289e-301, 3.0477082921762745e-274, 1.2958548079069575e-298, 1.6216610405068207e-205, 3.2947058013528015e-193, 0.0, 6.6928638907703034e-279, 1.24e-322, 2.085978e-317, 2.5789222671738825e-298, 8.48

[36.26598682040534, 76.06808100001417, 80.58121036444706, 8.582416264396375, 1.2035358458118446e-05, 7.250952231828416e-08, 2.630737781016818, 0.00967796252491402, 14.338821232767401, 62.7758022056434, 1.3126576861970023e-47, 1.8227479256222028, 0.03231435352530032, 214.6095461210403, 135.41558132407297, 1.999516699412626e-17, 33.142433861976166, 1.1248708025388456, 0.005837235683007498, 10.023650620611356, 0.6372769793242059, 194.57404484546652, 0.0053052947059168114, 2.328531128599712e-51, 6.953590909082867e-20, 192.28513380504117, 1.575994207547434e-13, 0.0001656340822192793, 0.0028044449149080206, 4.991221248274916, 2.3318394031186553, 35.05911842430358, 6.118322928899744e-16, 0.1801092913848634, 2.237213632526921, 227.44338464849855, 0.16202052237613004, 0.013087186814433882, 58.00283504247921, 77.54759747887445, 0.6321143821434826, 0.7330424974491427, 111.71395213658724, 226.835288302511, 1.5668574697363145, 3.9407363712547194e-08, 227.22017845822592, 1.0152742071486252e-31, 157.